In [31]:
# Imports
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [32]:
# create fully-connected network
class NN(nn.Module):
  def __init__(self, input_size, num_class):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_class)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [33]:
model = NN(784, 10)
x = torch.randn(64, 784)   # Batch, input
print(model(x).shape)

torch.Size([64, 10])


In [34]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
# Hyper Parameters
input_size = 784
num_class = 10
learning_rate = 0.001
batch_size= 64
num_epoches = 1

In [36]:
# Load Data
train_data = datasets.MNIST(root='dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle=True)

test_data = datasets.MNIST(root='dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle=True)

In [37]:
# Initialize
model = NN(input_size=input_size, num_class = num_class).to(device)

In [38]:
# Loss and Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
# Training Network
for epoch in range(num_epoches):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)

    data = data.reshape(data.shape[0], -1)  #Flatten Data

    # forward 
    scores = model(data)
    loss = criterion(scores, target)

    # backeard
    optimizer.zero_grad()
    loss.backward()

    # GD
    optimizer.step()

In [42]:
# Accuracy

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x= x.to(device=device)
      y= y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)

      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)

    print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct) / float(num_samples) * 100:.2f}"
        )
    
    model.train()


In [43]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 55896 / 60000 with accuracy 93.16
Got 9285 / 10000 with accuracy 92.85
